**IMPORTANT INFORMATION:** Two models were left on purpose so we could compare the difference between the first bad one and the second one were more funtionalities were added

##########IMPORTS##############

In [1]:
import keras.backend as K
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from tensorflow import keras
from keras.layers.core.dropout import Dropout
from tensorflow.keras import layers
from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Rescaling
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.layers import BatchNormalization
from pandas import DataFrame
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab import drive
import matplotlib.pyplot as plt

###########GOOGLE DRIVE MOUNT#############

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###########KAGGLE INFORMATION#############

In [4]:
os.environ['KAGGLE_USERNAME'] = 'davidreto'  # Your Kaggle username
os.environ['KAGGLE_KEY'] = '505caa4aee2ecd6bc8ae5f457a190351'  # Your Kaggle API key
os.environ['URN'] = '6647000'  # Your URN: submissions without a URN will not count

##########DATASET DOWNLOAD###############

In [ ]:
!cp -r /content/gdrive/MyDrive/Colab_Notebooks/Coursework_dataset_COM2028_21-22.zip /content/Coursework_dataset_COM2028_21-22.zip
!unzip /content/Coursework_dataset_COM2028_21-22.zip  

##########ADDING EXTENSION AFTER THE ID TO GET THE PICTURE#################

In [ ]:

df = pd.read_csv('train.csv',dtype=str)
col = []
for i in df["Id"]:
 col.append(i+".jpg")
df["filename"] = col

df
  

#######################DATASET TREATING AND RETRIVAL#########################

In [ ]:
colour_mode = 'grayscale'
image_size = (28, 28)
batch_size = 100



checkpoint_callback = ModelCheckpoint(
    filepath='checkpoints',
    save_weights_only=True,  # Record only weights (see model.save_weights vs model.save)
    monitor='val_accuracy',  # Evaluate model using accuracy metric on the validation set
    mode='max',  # Determine best model based on 'max' of the 'val_accuracy' score
    save_best_only=True)  # Keep only the best-performing model (i.e. overwrite only when the model has improved)

train_datagen = ImageDataGenerator(
    validation_split =0.2)

train_dataset = train_datagen.flow_from_dataframe(
  dataframe=df,
  directory='train',
  color_mode=colour_mode,
  
  y_col="Cell type",
  x_col="filename",
  subset='training',
  seed=42,
  class_mode='categorical',  # There are 8 classes for this task.
  target_size=image_size, 
  batch_size=batch_size)


validation_dataset = train_datagen.flow_from_dataframe(
  dataframe=df,
  directory='train',
  color_mode=colour_mode,
  
  y_col="Cell type",
  x_col="filename",
  subset='validation',
  seed=42,
  class_mode='categorical',  # There are 8 classes for this task.
  target_size=image_size,
  batch_size=batch_size)

test_dataset = image_dataset_from_directory(
  'test',
  shuffle=False,  # Important: the test dataset order must match Kaggle!
  labels=None,  # Important: the test dataset has no labels...
  color_mode=colour_mode,
  image_size=image_size,
  batch_size=100)  # Don't batch the test samples.


###########BAD MODEL################

In [ ]:
badmodel = Sequential()

badmodel.add(Rescaling(1./255)) 

badmodel.add(layers.RandomFlip("horizontal_and_vertical"))
badmodel.add(layers.RandomRotation(0.2))

badmodel.add(Conv2D(16, kernel_size=3,activation="relu",input_shape=(28,28,1)))
badmodel.add(Conv2D(16, kernel_size=3,activation="relu", padding="same"))
badmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1)))

badmodel.add(Conv2D(32, kernel_size=3,activation="relu", padding="same"))
badmodel.add(Conv2D(32, kernel_size=3,activation="relu", padding="same"))

badmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1)))
badmodel.add(Dropout(0.2))

badmodel.add(Conv2D(64, kernel_size=3,activation="relu", padding="same"))
badmodel.add(Conv2D(64, kernel_size=3,activation="relu", padding="same"))
badmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1)))

badmodel.add(Dropout(0.2))

badmodel.add(Conv2D(128, kernel_size=3,activation="relu", padding="same"))
badmodel.add(Conv2D(128, kernel_size=3,activation="relu", padding="same"))
badmodel.add(MaxPooling2D(pool_size=(2, 2)))

badmodel.add(Conv2D(256, kernel_size=3,activation="relu", padding="same"))
badmodel.add(Conv2D(256, kernel_size=3,activation="relu", padding="same"))
badmodel.add(MaxPooling2D(pool_size=(2, 2)))


badmodel.add(Flatten())
badmodel.add(Dropout(0.5))
badmodel.add(Dense(256,activation="relu"))
badmodel.add(Dense(512,activation="relu"))
badmodel.add(Dense(8,activation="softmax"))

badmodel.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

# Now train the model...

# Set up checkpointing callback, called when training:


history = badmodel.fit(train_dataset, validation_data=validation_dataset, epochs=30,callbacks=[checkpoint_callback])

########## ACURACY AND LOSS PLOTING FOR BAD MODEL################

In [ ]:
history_dict = history.history
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(1,41)
plt.plot(epochs, loss_train, 'g', label='Training Accuracy')
plt.plot(epochs, loss_val, 'b', label='validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,41)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

#####################FINAL MODEL######################

In [ ]:

model = Sequential()

model.add(Rescaling(1./255)) 

model.add(layers.RandomFlip("horizontal_and_vertical"))
model.add(layers.RandomRotation(0.2))

model.add(Conv2D(16, kernel_size=3,activation="relu",input_shape=(28,28,1)))
model.add(Conv2D(16, kernel_size=3,activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1)))

model.add(Conv2D(32, kernel_size=3,activation="relu", padding="same"))
model.add(Conv2D(32, kernel_size=3,activation="relu", padding="same"))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1)))
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=3,activation="relu", padding="same"))
model.add(Conv2D(64, kernel_size=3,activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1)))

model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=3,activation="relu", padding="same"))
model.add(Conv2D(128, kernel_size=3,activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=3,activation="relu", padding="same"))
model.add(Conv2D(256, kernel_size=3,activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256,activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(8,activation="softmax"))
lr_reduce=keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=4,
    verbose=0,
    mode="auto",
    min_lr=0.00001,
)
model.compile(optimizer='adamax',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

# Now train the model...

# Set up checkpointing callback, called when training:


history = model.fit(train_dataset, validation_data=validation_dataset, epochs=40,callbacks=[checkpoint_callback,lr_reduce])

########ACURACY AND LOSS PLOTING FOR FINAL MODEL################

In [ ]:
history_dict = history.history
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(1,41)
plt.plot(epochs, loss_train, 'g', label='Training Accuracy')
plt.plot(epochs, loss_val, 'b', label='validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,41)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

##################################PREDICTIONS###########################


In [ ]:
kidney_array = []
i=0
# Note:if you want to test the bad model you would need to change from model.predict to badmodel.predict
predictions = model.predict(test_dataset)

while i <  len(predictions):
  if np.argmax(predictions[i])==0 :
    kidney_array.append("0")

  elif np.argmax(predictions[i])==1 :
    kidney_array.append("1")    

  elif np.argmax(predictions[i])==2 :
    kidney_array.append("2")

  elif np.argmax(predictions[i])==3 :
    kidney_array.append("3")

  elif np.argmax(predictions[i])==4 :
    kidney_array.append("4")

  elif np.argmax(predictions[i])==5 :
    kidney_array.append("5")

  elif np.argmax(predictions[i])==6 :
    kidney_array.append("6")

  elif np.argmax(predictions[i])==7 :
    kidney_array.append("7")

  i=i+1
DataFrame(data={'Cell type': kidney_array}).to_csv('predictions.csv', index_label='Id')

######################KAGLE SUBMISSION#####################

In [ ]:
!python3 /usr/local/bin/kaggle competitions submit -m $URN -c uos-com2028-21-22-cw -f predictions.csv